# SBP - Locosys SiRF Binary (Packed)

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/gps-wizard/

In [1]:
import os
import sys

from datetime import datetime, timedelta, timezone

import numpy as np

projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

## SBP File Class

In [2]:
class SbpFile():
    '''SBP track - Locosys SiRF Binary (Packed)'''
    
    def __init__(self, filename):
        '''Basic init just reads the file into memory'''

        self.filename = filename

        with open(self.filename, "rb") as f:
            self.readHeader(f)
            self.readData(f)
            self.consumeData()
            
            
    def readHeader(self, f):
        '''Load header into memory'''
        
        self.header = {}

        header = f.read(64)
            
        # External length excludes itself but includes the start token (a0a2) + checksum + end token (b0b3)
        length = header[1] << 8 | header[0]
        if length > 62:
            raise ValueError('Invalid length {}'.format(length))

        # Check the start token is a0a2
        startToken = header[2] << 8 | header[3]
        if startToken != 0xa0a2:
            raise ValueError('Unexpected start token {:x} - expected {:x}'.format(startToken, 0xa0a2))

        # Internal length is sometimes big endian, sometimes little endian!
        if header[4] == 0:
            lengthAlt = header[5]
        else:
            lengthAlt = header[4]

        # Internal length excludes the start token (a0a2) + itself + checksum + end token (b0b3)
        if lengthAlt != length - 8:
            raise ValueError('Unexpected internal length {} (external length {})'.format(lengthAlt, length))

        # Main content
        content = header[6 : length - 2]
        
        # Parse ASCII data, ignoring content[0] which is the mid-file ID
        splitContent = content[1:].decode('ascii').split(',')
        self.header['username'] = splitContent[0]
        self.header['serial'] = int(splitContent[1])
        self.header['logRate'] = splitContent[2]
        self.header['firmware'] = splitContent[3]

        # Evaluate checksum
        checksum = header[length - 2] << 8 | header[length - 1]
        if checksum != sum(content):
            raise ValueError('Checksum difference - {}'.format(checksum - sum(content)))

        # Check the end token is b0b3
        endToken = header[length] << 8 | header[length + 1]
        if endToken != 0xb0b3:
            raise ValueError('Unexpected end token {:x} - expected {:x}'.format(endToken, 0xb0b3))
           
            
    def readData(self, f):
        '''Read data into memory'''
        
        fileSize = os.path.getsize(self.filename)
        
        if fileSize % 32 != 0:
            raise ValueError('File is not a multiple of 32 bytes')
        
        numRecords = (fileSize - 64) // 32

        dataType = np.dtype([
            ('hdop', 'u1'),
            ('sv_count', 'u1'),
            ('utc_secs', '<u2'),
            ('date_time_utc_packed', '<u4'),
            ('sv_ids', '>u4'),
            ('lat', '<i4'),
            ('lon', '<i4'),
            ('alt', '<i4'),
            ('sog', '<u2'),
            ('cog', '<u2'),
            ('climb_rate', '<i2'),
            ('sdop', 'u1'),
            ('vsdop', 'u1')
        ])

        self.rawData = np.fromfile(f, dtype=dataType, count=numRecords)



    def consumeData(self):
        '''Consume data - convert to standard data structure'''
        
        self.data = {}
        
        # Convert HDOP to decimal
        self.data['hdop'] = self.rawData['hdop'] / 5
        
        # Satellite count is already an integer
        self.data['sat'] = self.rawData['sv_count']
        
        # Convert time to milliseconds
        secs = self.rawData['utc_secs'] // 1000

        # Convert packed date/time to regular timestamp
        vfunc = np.vectorize(decodePackedDateTime)
        timestampSecs = vfunc(self.rawData['date_time_utc_packed'])
        
        # Check that sec and timestamp are consistent
        maxDiff = np.abs(secs - timestampSecs % 60).max()
        if maxDiff > 0:
            raise ValueError('Mismatch of seconds and timestamps')
        
        # Convert timestamp to milliseconds since the epoch
        self.data['timestamp'] = timestampSecs * 1000 + self.rawData['utc_secs'] % 1000

        # Skip satellite IDs
        
        # Convert latitude and longitude to decimal
        self.data['lat'] = self.rawData['lat'] / 10000000
        self.data['lon'] = self.rawData['lon'] / 10000000
        
        # Convert altitude and SOG from cm/s to mm/s
        self.data['alt'] = self.rawData['alt'] * 10
        self.data['sog'] = self.rawData['sog'] * 10
        
        # Convert COG to decimal
        self.data['cog'] = self.rawData['cog'] / 100

        # Convert climb rate, SDOP and VSDOP from cm/s to mm/s
        self.data['climb_rate'] = self.rawData['climb_rate'] * 10
        self.data['sdop'] = self.rawData['sdop'] * 10
        self.data['vsdop'] = self.rawData['vsdop'] * 10

        
def decodePackedDateTime(packedDateTime):
    '''Decode packed date/time'''

    second = packedDateTime & 0x3f      
    packedDateTime >>= 6

    minute = packedDateTime & 0x3f
    packedDateTime >>= 6

    hour = packedDateTime & 0x1f
    packedDateTime >>= 5

    day = packedDateTime & 0x1f
    packedDateTime >>= 5

    month = packedDateTime % 12
    year = packedDateTime // 12 + 2000

    dt = datetime(year, month, day, hour, minute, second)
    timestamp = int((dt - datetime(1970, 1, 1)) / timedelta(seconds=1))

    return timestamp

## Run Tests

In [3]:
if __name__ == '__main__':
    filename = os.path.join(projdir, 'sessions', '20220410', 'GW52_1Hz_K888_155800017_20220410_230017.sbp')
    sbpFile = SbpFile(filename)
    
    filename = os.path.join(projdir, 'sessions', '20220410', 'GW60_5Hz_JO_168601682_20220410_230113.sbp')
    sbpFile = SbpFile(filename)
    
    print('All done!')

All done!
